In [1]:
%run header_data_treatment.ipynb

In [2]:
works_au_af = spark.read.format('parquet').load('file:\\' + openalex_path + 'works_au_af.parquet')
df_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + 'publications.parquet')
df_au_scanR = spark.read.format('parquet').load('file:\\' + scanR_path + 'authors.parquet')

merge_path = 'D:\\openalex-snapshot\\merge\\'

merge_pub = spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr.parquet')
merge_authors = spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr_authors.parquet').withColumnRenamed('id','idref')


In [3]:
merge_authors = merge_authors.withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', ''))

In [4]:
expr_language ='CASE '+' '.join(
    ['WHEN title.' + field + ' IS NOT NULL THEN "'+ field + '"' for field in df_scanR.select('title.*').columns if field != 'default']) + 'ELSE NULL  END'


In [5]:
df_authors = (spark.read.format('parquet').load('file:\\' + openalex_path + 'authors.parquet')
              .withColumn('split_name', func.split(func.col('display_name'), ' '))
             .withColumn('first_name', func.element_at(func.col('split_name'), 1))
              .withColumn('last_name', func.concat_ws(' ', func.array_except(func.col('split_name'), func.array(func.col('first_name')))))
             )


In [6]:
works_au_af = (works_au_af
               .join(works_au_af
                     .filter((func.col('country').isNull()) 
                             | (func.col('country')=='FR'))
                     .select('work_id'), on = 'work_id', how = 'inner')
              )

In [ ]:
works_au_af_w_names = (works_au_af
                      .join(df_authors
                      .withColumn('author_id', func.regexp_replace(func.col('id'), 'https://openalex.org/',''))
                      .select('author_id', 'display_name', 'first_name', 'last_name'), on = ['author_id'], how = 'left')
                      )
works_au_af_w_names.cache()
works_au_af_w_names.show()

In [ ]:
df_scanR_exploded = (df_scanR
                     .withColumn('language', func.expr(expr_language))
                     .withColumn('authors', func.explode(func.col('authors')))
                     .withColumn('idref', func.regexp_replace(func.lower(func.col("authors.person")), 'idref',''))
                     .withColumn('firstName', func.regexp_replace(func.col('authors.firstName'), '\.', ''))
                     .withColumn('lastName', func.regexp_replace(func.col('authors .lastName'), '\.', ''))
                     .withColumn('fullName', func.regexp_replace(func.col('authors.fullName'), '\.', ''))
                     .withColumn('initial', func.substring(func.col('firstName'),1,1))
                     .withColumn('idref', func.when((func.col('idref').isNull()) & (func.col('productionType')!= 'thesis'), 
                                                func.col('fullName'))
                            .when( (func.col('idref').isNull()) & (func.col('productionType')=='thesis'),
                                                                    func.first(func.col('idref'), ignorenulls = True).over(Window.partitionBy('lastName','initial')))
                             .otherwise(func.col('idref')))
                     .withColumn('title', func.col('title.default'))
                    )

df_scanR_exploded.cache()
df_scanR_exploded.show()

In [ ]:
df_scanR.printSchema()

In [ ]:
import unidecode
def unidecode_debug(x):
    try:
        y = unidecode.unidecode(x)
        return(y)
    except:
        return(x)

udf_unidecode = func.udf(unidecode_debug, StringType())

import nltk
from nltk.corpus import stopwords
print(stopwords.fileids())
stopwords_dict = {}
for language in stopwords.fileids():
    stopwords_dict[language.capitalize()] = stopwords.words(language)

values = [(k, stopwords_dict[k]) for k in list(stopwords_dict.keys())]
columns = ['language', 'stopwords']
stopwords_df = spark.createDataFrame(values, columns)

iso_639_choices ={
'ab': 'Abkhaz', 'aa': 'Afar', 'af': 'Afrikaans', 'ak': 'Akan', 'sq': 'Albanian', 'am': 'Amharic', 'ar': 'Arabic',
'an': 'Aragonese', 'hy': 'Armenian', 'as': 'Assamese', 'av': 'Avaric', 'ae': 'Avestan', 'ay': 'Aymara', 'az': 'Azerbaijani',
'bm': 'Bambara', 'ba': 'Bashkir', 'eu': 'Basque', 'be': 'Belarusian', 'bn': 'Bengali', 'bh': 'Bihari', 'bi': 'Bislama',
'bs': 'Bosnian', 'br': 'Breton', 'bg': 'Bulgarian', 'my': 'Burmese', 'ca': 'Catalan; Valencian', 'ch': 'Chamorro', 'ce': 'Chechen',
'ny': 'Chichewa; Chewa; Nyanja', 'zh': 'Chinese', 'cv': 'Chuvash', 'kw': 'Cornish', 'co': 'Corsican', 'cr': 'Cree', 'hr': 'Croatian',
'cs': 'Czech', 'da': 'Danish', 'dv': 'Divehi; Maldivian;', 'nl': 'Dutch', 'dz': 'Dzongkha', 'en': 'English', 'eo': 'Esperanto',
'et': 'Estonian', 'ee': 'Ewe', 'fo': 'Faroese', 'fj': 'Fijian', 'fi': 'Finnish', 'fr': 'French', 'ff': 'Fula', 'gl': 'Galician',
'ka': 'Georgian', 'de': 'German', 'el': 'Greek', 'gn': 'Guaraní', 'gu': 'Gujarati', 'ht': 'Haitian','ha': 'Hausa',
'he': 'Hebrew','hz': 'Herero','hi': 'Hindi','hi': 'Hinglish','ho': 'Hiri Motu','hu': 'Hungarian','ia': 'Interlingua','id': 'Indonesian',
'ie': 'Interlingue','ga': 'Irish','ig': 'Igbo','ik': 'Inupiaq','io': 'Ido','is': 'Icelandic','it': 'Italian','iu': 'Inuktitut',
'ja': 'Japanese','jv': 'Javanese', 'kl': 'Kalaallisut', 'kn': 'Kannada', 'kr': 'Kanuri', 'ks': 'Kashmiri', 
'kk': 'Kazakh', 'km': 'Khmer', 'ki': 'Kikuyu, Gikuyu', 'rw': 'Kinyarwanda', 'ky': 'Kirghiz, Kyrgyz', 'kv': 'Komi', 
'kg': 'Kongo', 'ko': 'Korean', 'ku': 'Kurdish', 'kj': 'Kwanyama, Kuanyama', 'la': 'Latin', 'lb': 'Luxembourgish', 
'lg': 'Luganda', 'li': 'Limburgish', 'ln': 'Lingala', 'lo': 'Lao', 'lt': 'Lithuanian', 'lu': 'Luba-Katanga', 'lv': 'Latvian',
'gv': 'Manx', 'mk': 'Macedonian', 'mg': 'Malagasy', 'ms': 'Malay', 'ml': 'Malayalam', 'mt': 'Maltese', 'mi': 'Māori', 
'mr': 'Marathi','mh': 'Marshallese','mn': 'Mongolian','na': 'Nauru','nv': 'Navajo','nb': 'Norwegian Bokmål',
'nd': 'North Ndebele','ne': 'Nepali','ng': 'Ndonga','nn': 'Norwegian Nynorsk','no': 'Norwegian','ii': 'Nuosu',
'nr': 'South Ndebele','oc': 'Occitan','oj': 'Ojibwe, Ojibwa','cu': 'Old Church Slavonic','om': 'Oromo','or': 'Oriya',
'os': 'Ossetian, Ossetic','pa': 'Panjabi, Punjabi','pi': 'Pāli','fa': 'Persian','pl': 'Polish','ps': 'Pashto, Pushto',
'pt': 'Portuguese','qu': 'Quechua','rm': 'Romansh','rn': 'Kirundi', 'ro': 'Romanian', 'ru': 'Russian', 
'sa': 'Sanskrit (Saṁskṛta)', 'sc': 'Sardinian', 'sd': 'Sindhi', 'se': 'Northern Sami', 'sm': 'Samoan', 'sg': 'Sango',
'sr': 'Serbian', 'gd': 'Scottish Gaelic', 'sn': 'Shona', 'si': 'Sinhala, Sinhalese', 'sk': 'Slovak', 'sl': 'Slovene',
'so': 'Somali', 'st': 'Southern Sotho', 'es': 'Spanish', 'su': 'Sundanese', 'sw': 'Swahili', 'ss': 'Swati', 'sv': 'Swedish', 
'ta': 'Tamil', 'te': 'Telugu', 'tg': 'Tajik', 'th': 'Thai', 'ti': 'Tigrinya','bo': 'Tibetan','tk': 'Turkmen','tl': 'Tagalog','tn': 'Tswana','to': 'Tonga','tr': 'Turkish','ts': 'Tsonga','tt': 'Tatar','tw': 'Twi','ty': 'Tahitian','ug': 'Uighur, Uyghur','uk': 'Ukrainian','ur': 'Urdu','uz': 'Uzbek','ve': 'Venda','vi': 'Vietnamese','vo': 'Volapük','wa': 'Walloon','cy': 'Welsh','wo': 'Wolof','fy': 'Western Frisian','xh': 'Xhosa','yi': 'Yiddish','yo': 'Yoruba','za': 'Zhuang, Chuang',
'zu': 'Zulu', "zh-cn" : "Chinese"
}
        

values = [(k, iso_639_choices[k]) for k in list(iso_639_choices.keys())]
columns = ["language_iso2",'language']
iso_df = spark.createDataFrame(values, columns)
#iso_df.show()

stopwords_df = (stopwords_df.join(iso_df.withColumn('language', func.explode(func.split(func.col('language'), '; '))), on = 'language', how = 'left')
                .select('stopwords', func.col('language_iso2').alias('language'))
               )
def rm_stopwords(x,y):
    if x is None:
        return(x)
    if y is None:
        return(y)
    else:
        try:
            list_x = x.split(' ')
            list_y = y
            return(' '.join([word for word in list_x if word not in list_y]))
        except:
            return(x)
udf_rm_stopwords = func.udf(rm_stopwords)
#stopwords_df.show()

In [ ]:
def get_only_unmatched(base_1, base_2, merged, id_1, id_2):

    unmatched_1 = (base_1
                   .join(merged.select(id_1), on = id_1, how = 'leftanti')
                  )
    unmatched_2 = (base_2
                   .join(merged.select(id_2), on = id_2, how = 'leftanti')
                  )
    return(unmatched_1, unmatched_2)

In [ ]:
def match_authors_from_pub(merged, base_1, base_2, pub_id_1, pub_id_2,
                           au_id_1, au_id_2, last_name_1, last_name_2, name_1, name_2):

    to_merge_1 = (base_1
                .join(merged.select(pub_id_1), on = pub_id_1, how = 'inner')
                .select(au_id_1, func.substring(func.col(name_1), 1, 1).alias('initial'), func.col(last_name_1).alias('last_name'), pub_id_1)
               )
    #print(to_merge_1.count())
    to_merge_2 = (base_2
                .join(merged.select(pub_id_1, pub_id_2), on = pub_id_2, how = 'inner')
                .select(au_id_2, func.substring(func.col(name_2), 1, 1).alias('initial'), func.col(last_name_2).alias('last_name'), pub_id_1, pub_id_2)
                 )
    #print(to_merge_2.count())
    matched = (to_merge_1
               .join(to_merge_2, on = [pub_id_1, 'last_name', 'initial'], how = 'inner')
               .withColumn('n_diff_id_1', func.size(func.collect_set(func.col(au_id_1)).over(Window.partitionBy(au_id_2))))
               .withColumn('n_diff_id_2', func.size(func.collect_set(func.col(au_id_2)).over(Window.partitionBy(au_id_1))))
               .filter( (func.col('n_diff_id_1')==1) &  (func.col('n_diff_id_2')==1))
               .select(au_id_1, au_id_2)
               .distinct()
              )
    return(matched)

In [21]:
def match_pubs_from_authors(merged, base_1, base_2, pub_id_1, pub_id_2,
                           au_id_1, au_id_2, title):

    to_merge_1 = (base_1
                .join(merged.select(au_id_1), on = au_id_1, how = 'inner')
                .join(stopwords_df, on = ['language'], how = 'left')
                .withColumn('cleaned_title', udf_rm_stopwords(func.lower(func.col(title)), func.col('stopwords')))
                .withColumn('cleaned_title', udf_unidecode(func.col('cleaned_title')))
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), r'[[^A-Za-z0-9 -]]+', ''))  # Remove special characters
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), '  ', ''))  # Remove special characters
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), '  ', ''))  # Remove special characters
                .select(au_id_1, 'cleaned_title', pub_id_1)
               )
    #print(to_merge_1.count())
    to_merge_2 = (base_2
                .join(merged.select(au_id_1, au_id_2), on = au_id_2, how = 'inner')
                .join(stopwords_df, on = ['language'], how = 'left')
                .withColumn('cleaned_title', udf_rm_stopwords(func.lower(func.col(title)), func.col('stopwords')))
                .withColumn('cleaned_title', udf_unidecode(func.col('cleaned_title')))
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), r'[[^A-Za-z0-9 -]]+', ''))  # Remove special characters
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), '  ', ''))  # Remove special characters
                .withColumn('cleaned_title', func.regexp_replace(func.col('cleaned_title'), '  ', ''))  # Remove special characters
                .select(au_id_1, au_id_2, 'cleaned_title', pub_id_2)
                 )
    #print(to_merge_2.count())

    matched = (to_merge_1
               .join(to_merge_2, on = [au_id_1, 'cleaned_title'], how = 'inner')
               .withColumn('n_diff_id_1', func.size(func.collect_set(func.col(pub_id_1)).over(Window.partitionBy(pub_id_2))))
               .withColumn('n_diff_id_2', func.size(func.collect_set(func.col(pub_id_2)).over(Window.partitionBy(pub_id_1))))
               .filter( (func.col('n_diff_id_1')==1) &  (func.col('n_diff_id_2')==1))
               .select(pub_id_1, pub_id_2)
               .distinct()
              )
    return(matched)

In [40]:
merge_pub = spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr.parquet')
merge_pub.select('work_id','id').unionAll(new_match_pubs_from_au).show()

+-----------+--------------------+
|    work_id|                  id|
+-----------+--------------------+
|W2912446109|doi10.1001/amajet...|
|W2913698685|doi10.1001/jama.2...|
|W2911290113|doi10.1001/jama.2...|
|W2912431805|doi10.1001/jama.2...|
|W2936759375|doi10.1001/jama.2...|
|W2964642789|doi10.1001/jama.2...|
|W2969751565|doi10.1001/jama.2...|
|W2921906072|doi10.1001/jama.2...|
|W2988464799|doi10.1001/jama.2...|
|W2978101814|doi10.1001/jama.2...|
|W2978470254|doi10.1001/jama.2...|
|W2986540284|doi10.1001/jama.2...|
|W2995672851|doi10.1001/jama.2...|
|W2995170461|doi10.1001/jama.2...|
|W2996126894|doi10.1001/jama.2...|
|W2945858218|doi10.1001/jama.2...|
|W2942311206|doi10.1001/jama.2...|
|W2944420877|doi10.1001/jama.2...|
|W2946375103|doi10.1001/jama.2...|
|W2962938985|doi10.1001/jama.2...|
+-----------+--------------------+
only showing top 20 rows



In [28]:
iterations = 0
while iterations < 1:
    iterations = iterations +1
    print('Iteration: ', iterations + 1)
    if iterations == 1:
        merge_pub = spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr.parquet')
        merge_authors = (spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr_authors.parquet')
                     .withColumnRenamed('id','idref')
                     .withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', '')))
    else: 
        merge_pub = spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr_it.parquet')
        merge_authors = (spark.read.format('parquet').load('file:\\' + merge_path + 'merge_oa_scanr_authors_it.parquet')
                     .withColumnRenamed('id','idref')
                     .withColumn('idref', func.regexp_replace(func.col('idref'), 'idref', '')))

    print('Nr matches for publications: ', merge_pub.count(), ', nr matches for authors: ',    merge_authors.count())

    
    unmatched_pubs_oa, unmatched_pubs_scanR = get_only_unmatched(works_au_af_w_names, df_scanR_exploded, merge_pub, 'work_id', 'id')
    unmatched_au_oa, unmatched_au_scanR = get_only_unmatched(works_au_af_w_names, df_scanR_exploded, merge_authors, 'author_id', 'idref')

    print('ok')
    new_match_au_from_pubs = match_authors_from_pub(merge_pub, unmatched_au_oa, unmatched_au_scanR, 'work_id', 'id',
                           'author_id', 'idref', 'last_name', 'lastName', 'display_name', 'fullName')
    
    new_match_pubs_from_au = match_pubs_from_authors(merge_authors, unmatched_pubs_oa, unmatched_pubs_scanR, 'work_id', 'id',
                           'author_id', 'idref', 'title')

    merge_authors = (merge_authors.unionAll(new_match_au_from_pubs))
    merge_pub = (merge_pub.select('work_id', 'id').unionAll(new_match_pubs_from_au))

    merge_authors.count()
    merge_pub.count()
    merged_authors.write.mode('overwrite').parquet('file:\\' + merge_path + 'merge_oa_scanr_authors_it.parquet')
    merge_pub.write.mode('overwrite').parquet('file:\\' + merge_path + 'merge_oa_scanr_it.parquet')
    #print('New matches of authors from publications : ', new_match_au_from_pubs.count(), '\n',
    #      'New matches of publications from authors : ', new_match_pubs_from_au.count())
    

    

Iteration:  2
Nr matches for publications:  201792 , nr matches for authors:  414106
ok


In [34]:
new_match_pubs_from_au.show()

+-----------+--------------------+
|    work_id|                  id|
+-----------+--------------------+
|W1000137256|   halinria-00596868|
| W100014352|     nnt2003aix20651|
| W100015661|     nnt2009lyo10148|
| W100021887|     nnt2004mon20145|
|W1000261152|doi10.1016/j.msea...|
|W1000436346|  halhalshs-00777257|
|  W10006723|     nnt2011clf22104|
| W100076178|     nnt2010tou30278|
|W1000816400|doi10.1007/s13171...|
| W100088650|     nnt2009pa066033|
| W100093082|     nnt1999toul0006|
|W1000960367|   halinria-00549216|
|W1001064943|     halhal-01647440|
|W1001077819|doi10.1103/physre...|
|W1001091333|     nnt2013aixm4077|
|W1001093056|  halhalshs-00637916|
| W100110499|     nnt1987dijos036|
| W100113257|     nnt2011pa100093|
| W100113666|     nnt2004bor30070|
| W100162093|     halhal-00809316|
+-----------+--------------------+
only showing top 20 rows



In [43]:
merge_pub_new = merge_pub.select('work_id','id').unionAll(new_match_pubs_from_au)

In [44]:
merge_pub_new.show()

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "C:\Users\rapha\anaconda3\Lib\socket.py", line 709, in readinto
    raise
TimeoutError: timed out


In [30]:
print('Nr matches for publications: ', merge_pub.count())


Py4JJavaError: An error occurred while calling o3668.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 99 in stage 283.0 failed 1 times, most recent failure: Lost task 99.0 in stage 283.0 (TID 3820) (localhost executor driver): java.net.SocketException: Connection reset by peer
	at java.base/sun.nio.ch.SocketDispatcher.write0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.write(SocketDispatcher.java:54)
	at java.base/sun.nio.ch.NioSocketImpl.tryWrite(NioSocketImpl.java:394)
	at java.base/sun.nio.ch.NioSocketImpl.implWrite(NioSocketImpl.java:413)
	at java.base/sun.nio.ch.NioSocketImpl.write(NioSocketImpl.java:440)
	at java.base/sun.nio.ch.NioSocketImpl$2.write(NioSocketImpl.java:819)
	at java.base/java.net.Socket$SocketOutputStream.write(Socket.java:1195)
	at java.base/java.io.BufferedOutputStream.implWrite(BufferedOutputStream.java:217)
	at java.base/java.io.BufferedOutputStream.write(BufferedOutputStream.java:200)
	at java.base/java.i

+--------------+
|         idref|
+--------------+
|idref108538427|
|idref225993244|
|idref17820806X|
|idref097879134|
|idref079013848|
|idref27199214X|
|idref117817007|
|idref258447958|
|idref140407960|
|idref221650709|
|idref081786883|
|idref078649781|
|idref178968315|
|idref250398206|
|idref25101651X|
|idref165190132|
|idref175449716|
|idref232567670|
|idref116414014|
|idref196687144|
+--------------+
only showing top 20 rows



In [16]:
df_scanR_exploded.select('idref').show()

+--------------------+
|               idref|
+--------------------+
|Jonathan A Christ...|
| Raymond A Ezejiofor|
|   A K Rakhmetullina|
|     Hatem A Rashwan|
|     Larissa A Rolim|
|     Habibah A Wahab|
|   Gabriel AB Marais|
|                NULL|
|    AHMAD ABD‐ALAZIZ|
|           124735150|
|                NULL|
|             A ADDED|
|        NEZAM AFDHAL|
|             D AGIUS|
|           178752061|
|                NULL|
|                NULL|
|                NULL|
|           086036904|
|           086036904|
+--------------------+
only showing top 20 rows



In [11]:
unmatched_pubs_oa, unmatched_pubs_scanR = get_only_unmatched(works_au_af_w_names, df_scanR_exploded, merge_pub, 'work_id', 'id')

In [12]:
unmatched_au_oa, unmatched_au_scanR = get_only_unmatched(works_au_af_w_names, df_scanR_exploded, merge_authors, 'author_id', 'idref')

In [13]:
match_pubs_from_authors(merge_authors, unmatched_pubs_oa, unmatched_pubs_scanR, 'work_id', 'id',
                           'author_id', 'idref', 'title').show()

38338654
0


DataFrame[work_id: string, id: string]

In [14]:
match_pubs_from_authors(merge_authors, unmatched_pubs_oa, unmatched_pubs_scanR, 'work_id', 'id',
                           'author_id', 'idref', 'title').show()

38338654
0
+-------+---+
|work_id| id|
+-------+---+
+-------+---+



In [36]:
merge_authors[].show()

+--------------+-----------+
|         idref|  author_id|
+--------------+-----------+
|idref108538427|A5080431922|
|idref225993244|A5009476002|
|idref17820806X|A5045317380|
|idref097879134|A5043440547|
|idref079013848|A5091569743|
|idref27199214X|A5073550025|
|idref117817007|A5079720434|
|idref258447958|A5018502157|
|idref140407960|A5007698196|
|idref221650709|A5018830374|
|idref081786883|A5040388573|
|idref078649781|A5065112242|
|idref178968315|A5005892790|
|idref250398206|A5078787536|
|idref25101651X|A5072381169|
|idref165190132|A5028531834|
|idref175449716|A5009278937|
|idref232567670|A5079074755|
|idref116414014|A5002320268|
|idref196687144|A5023926454|
+--------------+-----------+
only showing top 20 rows



In [37]:
df_scanR_exploded.select('idref').filter(func.col('idref').rlike('\d')).show()

+--------------------+
|               idref|
+--------------------+
|076086224###g.maz...|
|189607939###ouria...|
|           Alice M 1|
|Achille Cesare Pe...|
|           Geloen 12|
|Holger S Willenbe...|
|             Huot 14|
|Vila 1exmulri Mag...|
|            342 Choi|
|       457 Borgonovo|
|  Leo Schultzekool 7|
|145470768###aurél...|
|076926656###aurél...|
|169207226###renau...|
|13028906x###tara ...|
|13028906x###tara ...|
|13028906x###tara ...|
|13028906x###tara ...|
|13028906x###tara ...|
|13028906x###tara ...|
+--------------------+
only showing top 20 rows



In [24]:
(unmatched_au_scanR
                .join(merge_pub.select('work_id', 'id'), on = 'id', how = 'inner')
                .select('idref', func.substring(func.col('fullName'), 1, 1).alias('initial'), func.col('lastName').alias('last_name'), 'id','work_id')
                 ).show()

+--------------------+-------+--------------+--------------------+
|               idref|initial|     last_name|                  id|
+--------------------+-------+--------------+--------------------+
|  Alain Palloix ( ‡)|      A|          ( ‡)|doi10.25518/1780-...|
|              Goel A|      G|             A|doi10.16965/ijar....|
|            Gevrey A|      G|             A|     halhal-02304673|
|            Gevrey A|      G|             A|     halhal-02304776|
|           Garnier A|      G|             A|     halhal-03566942|
|          Harroche A|      H|             A|doi10.31031/rpn.2...|
|         Houeijeh, A|      H|             A|     halhal-02466637|
|           Lemaire A|      L|             A|doi10.16966/2378-...|
|     Leitenstorfer A|      L|             A|doi10.1051/epjcon...|
|  Lacorte-Bruguera A|      L|             A|     halhal-02332778|
|      A Myrkassimova|      A|A Myrkassimova|doi10.32014/2019....|
|     Larissa A Rolim|      L|       A Rolim|doi10.21577/1984-